In [1]:
import tensorflow as tf
import numpy as np

In [2]:
num_points = 2000
vectors_set = []
for i in xrange(num_points):
    if np.random.random() > 0.5 :
        vectors_set.append([np.random.normal(0.0, 0.9), np.random.normal(0.0,0.9)])
    else:
        vectors_set.append([np.random.normal(3.0, 0.5), np.random.normal(1.0,0.5)])

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

df = pd.DataFrame({"x":[v[0] for v in vectors_set],"y":[v[1] for v in vectors_set]})
sns.lmplot("x","y",data=df,fit_reg=False,size=6)
plt.show()

//anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
//anaconda/lib/python2.7/site-packages/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


상수 텐서를 작성 

In [4]:
vectors = tf.constant(vectors_set)   

중심값을 설정

In [5]:
k = 4
centroides = tf.Variable(tf.slice(tf.random_shuffle(vectors),[0,0],[k,-1]))

두 차원의 크기를 맞춤<br>
vectors와 centroides 두 변수의 차이를 구합니다. <br>
이 차이값으로 새로운 클러스터를 결정합니다. 

In [ ]:
expanded_vectors = tf.expand_dims(vectors,0) #vectors : 2
expanded_centroides = tf.expand_dims(centroides,1)

유클리드 제곱거리를 사용하는 할당단계의 알고리즘

In [ ]:
assignments = tf.argmin(tf.reduce_sum(tf.square(tf.subtract(expanded_vectors,expanded_centroides)),2),0)

풀어 쓰면 <br>
diff = tf.substract(expanded_vectors, expanded_centroides)<br>
sqr =  tf.square(diff)<br>
distances = tf.reduce_sum(sqr,2)<br>
assignments = tf.argmin(distances, 0)<br>

새로운 중심 계산

K개 군집에 속하는 점들의 평균을 가진 K개의 텐서를 합쳐서 means 텐서를 생성

In [ ]:
means = tf.concat([tf.reduce_mean(tf.gather(vectors,tf.reshape(tf.where(tf.equal(assignments,c)),[1,-1])), reduction_indices=[1]) for c in xrange(k)],0)

<h1>그래프 실행</h1>

means 텐서 값을 갱신 해야 다음번 수행에서 사용할수 있음 <br>
이를 위해 centroides에 means 값을 할당

In [ ]:
update_centroides = tf.assign(centroides, means)

데이터 그래프를 실행하기 위해서 모든 변수를 초기화

In [ ]:
init_op = tf.global_variables_initializer()

데이터 그래프를 실행

In [ ]:
sess = tf.Session()
sess.run(init_op)

for step in xrange(100):
    _, centroid_values, assignment_values = sess.run([update_centroides, centroides, assignments])
    print centroid_values

In [ ]:
data = {"x":[], "y":[], "cluster":[]}

for i in xrange(len(assignment_values)):
    data["x"].append(vectors_set[i][0])
    data["y"].append(vectors_set[i][1])
    data["cluster"].append(assignment_values[i])
    
df = pd.DataFrame(data)
sns.lmplot("x","y", data=df, fit_reg=False, size=6, hue="cluster", legend=False)
plt.show()